# Tensor flow 

## Load Dataset

In [1]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
print(sys.path)

['/Users/thiradatiamklang/Desktop/flight-streamlit-at3/flight-prediction/notebooks/TT_notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages']


In [3]:
import sys
sys.path.append('../../src')


In [4]:
from data.make_dataset import load_sets
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../../data/processed/')

In [5]:
import pandas as pd
# Read a Feather file
df = pd.read_feather('../../data/processed/df_cleaned_select_cols.feather')

In [6]:
from data.make_dataset import pop_target
features, target = pop_target(df, 'totalFare')

In [7]:
y_train

array([-0.31391783, -0.50182474, -1.22936172, ..., -0.89209292,
        1.17970114, -0.04892092])

In [8]:
X_train

array([[ 1.07916650e+07, -9.95738027e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.53247500e+06, -7.88605694e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.91918400e+06, -5.74075777e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.18766450e+07, -4.60646166e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.16629820e+07,  4.85011571e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.58366700e+06,  1.26422368e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

## TensorFlow's Decision Forests (TF-DF)

In [14]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from data.make_dataset import pd_dataframe_to_tf_dataset

# Define batch size
batch_size = 32

# Convert data to tf.data.Datasets
train_dataset = pd_dataframe_to_tf_dataset(X_train, y_train, batch_size)
validation_dataset = pd_dataframe_to_tf_dataset(X_val, y_val, batch_size)
test_dataset = pd_dataframe_to_tf_dataset(X_test, y_test, batch_size)

ImportError: cannot import name 'pd_dataframe_to_tf_dataset' from 'data.make_dataset' (/Users/thiradatiamklang/Desktop/flight-streamlit-at3/flight-prediction/notebooks/TT_notebooks/../../src/data/make_dataset.py)

### 1. TensorFlow Decision Forests (TF-DF) models

In [42]:
# Define the TensorFlow Decision Forest model
model = tfdf.keras.RandomForestModel()

# Compile the model
model.compile(metrics=["mean_squared_error"])

# Fit the model
model.fit(train_input, epochs=100, validation_data=validation_input)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/sb/nxrzyd0n61192x17k7wcyr4w0000gn/T/ipykernel_55045/1784112371.py", line 2, in <module>
    train_input = tfdf.keras.pd_dataframe_to_tf_dataset(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 1091, in pd_dataframe_to_tf_dataset
    dataframe = dataframe.copy(deep=True)
TypeError: copy() got an unexpected keyword argument 'deep'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File 

In [ ]:
# Evaluate the model
results = model.evaluate(test_input)

# Print evaluation results
print(f'Mean Squared Error: {results["mean_squared_error"]}')

# Make predictions
predictions = model.predict(test_input)

### 2. Random Forest models

In [ ]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf

# Define and train the Random Forest models
model_rf1 = tfdf.keras.RandomForestModel(task=task)
model_rf2 = tfdf.keras.RandomForestModel(
    task=task, num_trees=100, max_depth=10, split_axis="SPF_OHE"
)

model_rf1.fit(train_input)
model_rf2.fit(train_input)

# Evaluate the Random Forest models
results_rf1 = model_rf1.evaluate(test_input)
results_rf2 = model_rf2.evaluate(test_input)

# Print evaluation results for Random Forest
print("Random Forest Model 1 Evaluation:")
print(results_rf1)
print("Random Forest Model 2 Evaluation:")
print(results_rf2)

# Define and train the Gradient Boosted Trees models
model_gbdt1 = tfdf.keras.GradientBoostedTreesModel(task=task)
model_gbdt2 = tfdf.keras.GradientBoostedTreesModel(
    task=task, num_trees=100, growing_strategy="BEST_FIRST_GLOBAL"
)

model_gbdt1.fit(train_input)
model_gbdt2.fit(train_input)

# Evaluate the Gradient Boosted Trees models
results_gbdt1 = model_gbdt1.evaluate(test_input)
results_gbdt2 = model_gbdt2.evaluate(test_input)

# Print evaluation results for Gradient Boosted Trees
print("Gradient Boosted Trees Model 1 Evaluation:")
print(results_gbdt1)
print("Gradient Boosted Trees Model 2 Evaluation:")
print(results_gbdt2)


### 3. Gradient Boosted Trees